<a href="https://colab.research.google.com/github/surendrallam/consolidate-banking-statment/blob/main/Simple_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [ ]:
%tensorflow_version 2.x
import sys, time
sys.path[3] = '/usr/lib/python36.zip'
sys.path[4] = '/usr/lib/python3.6'
sys.path[5] = '/usr/lib/python3.6/lib-dynload'
sys.path[6] = '/usr/local/lib/python3.6/dist-packages'
sys.path[8] = '/usr/local/lib/python3.6/dist-packages/IPython/extensions'
!pip install tensorflow==2.4.1
!pip install keras==2.4.0
from IPython.display import clear_output

time.sleep(3)
clear_output()
time.sleep(1)

# Restart the process to effect the changes
import os
os.kill(os.getpid(), 9) #(This automatically restarts the runtime so the changes take effect)

In [25]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU, GlobalAveragePooling2D
import numpy as np

In [26]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

In [27]:
print("Tensorflow",tf.__version__) #(This is just to check that everything went right)
print("Keras",keras.__version__)

Tensorflow 2.4.1
Keras 2.4.0


In [28]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28 

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000,28,28,1) # (60000,28,28,1) 224,224
x_test = x_test.reshape(10000,28,28,1)   # (10000,28,28,1)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)
60000 train samples
10000 test samples


Layer constructing

In [31]:
model = Sequential()

# Default 
# model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))

# MRD
model.add(Conv2D(32, kernel_size=(3, 3),  strides=(2, 2), activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(GlobalAveragePooling2D(name='avg_pool'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 13, 13, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 4, 4, 64)          18496     
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 64)                0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
Total params: 19,466
Trainable params: 19,466
Non-trainable params: 0
__________________________________________________

Model Training

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

EPOCHS = 10
BATCH_SIZE = 32
STEPS_PER_EPOCH = 10
VALIDATION_STEPS = 32
 

model.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          steps_per_epoch=STEPS_PER_EPOCH,
          verbose=1,
          validation_data=(x_test, y_test),
          validation_steps=VALIDATION_STEPS)

score = model.evaluate(x_test, y_test, verbose=2)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print(score[2])

Saving the model

In [9]:
MODEL_FILE = 'mobilenet/filename.model'

model.save(MODEL_FILE)
backup_model = model
# model.summary()

full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(x=tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
 

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()
 
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)

 
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
 logdir="mobilenet/frozen_models",
 name="simple_mrd_cnn_ker_2.4.0.pb",
 as_text=False)
 
tf.saved_model.save(model, "mobilenet/simple_model")
print("Done")

--------------------------------------------------
Done


### Reference
* https://towardsdatascience.com/build-your-own-convolution-neural-network-in-5-mins-4217c2cf964f